In [14]:
# ======================================================================
# CELL 3 — Locate your Poshan file (auto-detect extension)
# ======================================================================
def find_poshan_file(data_dir: str, base: str):
    for ext in (".csv", ".xlsx", ".xls"):
        cand = os.path.join(data_dir, base + ext)
        if os.path.exists(cand):
            return cand
    matches = glob(os.path.join(data_dir, base + ".*"))
    if matches:
        return matches[0]
    raise FileNotFoundError(f"Could not find {base}.csv/.xlsx/.xls in {data_dir}")

DATA_PATH = find_poshan_file(DATA_DIR, DATA_BASENAME)
print("Using file:", DATA_PATH)

Using file: C:\Users\JOSHUVA\OneDrive\Desktop\Projects\UNICEF\mapped_poshan_tracker_updated.csv


In [16]:
# ======================================================================
# CELL 4 — Robust Poshan Loader (handles LONG or WIDE; infers month/year/date)
# Output schema: date | state | district | indicator | value | unit
# ======================================================================
MONTH_NAME_MAP = {m.lower(): i for i, m in enumerate(calendar.month_name) if m}
MONTH_ABBR_MAP = {m.lower(): i for i, m in enumerate(calendar.month_abbr) if m}

def parse_month(m):
    """Accept 1..12 or names/abbr (Jan/January). Return int month or NaN."""
    if pd.isna(m): return np.nan
    try:
        v = int(str(m).strip())
        return v if 1 <= v <= 12 else np.nan
    except:
        s = str(m).strip().lower()
        if s in MONTH_NAME_MAP: return MONTH_NAME_MAP[s]
        if s in MONTH_ABBR_MAP: return MONTH_ABBR_MAP[s]
        # tolerate "YYYY-MM" style in a month field
        if re.match(r"^\d{4}[-/]\d{1,2}$", s):
            try:
                return int(s.split("-")[-1].split("/")[-1])
            except: return np.nan
        return np.nan

def to_month_start(y, m):
    y = int(y); m = int(m)
    return pd.Timestamp(year=y, month=m, day=1)

def _is_numeric_series(s: pd.Series, min_non_na=0.5):
    s_num = pd.to_numeric(s, errors="coerce")
    return ((~s_num.isna()).mean() >= min_non_na)

def _infer_unit_from_indicator_and_values(ind_name: str, values: pd.Series):
    name = (ind_name or "").lower()
    if "%" in name or "percent" in name or "pct" in name:
        return "%"
    vals = pd.to_numeric(values, errors="coerce")
    if vals.dropna().between(0, 100).mean() > 0.9:
        return "%"
    return ""  # counts or unknown
def load_poshan_any(path: str) -> pd.DataFrame:
    # 1) Read file
    ext = Path(path).suffix.lower()
    if ext in [".xlsx", ".xls"]:
        df = pd.read_excel(path, engine="openpyxl")
    else:
        df = pd.read_csv(path)

    # 2) Normalize headers
    def norm(c): return re.sub(r"\s+", " ", str(c)).strip()
    df = df.rename(columns={c: norm(c) for c in df.columns})

    # 3) Identify id-like columns
    id_map = {}
    for c in df.columns:
        lc = c.lower()
        if any(k in lc for k in ["state", "state_name"]):         id_map.setdefault("state", c)
        if any(k in lc for k in ["dist", "district"]):            id_map.setdefault("district", c)
        if lc in ("year","fyyear","fiscal_year","yr"):            id_map.setdefault("year", c)
        if any(k in lc for k in ["month","mnth","mth","monthname"]): id_map.setdefault("month", c)
        if "date" in lc and "update" not in lc:                   id_map.setdefault("date", c)
        if any(k in lc for k in ["unit","units","uom"]):          id_map.setdefault("unit", c)

    # 4) Try LONG format detection (indicator + value exist)
    indicator_col = None
    value_col = None
    for c in df.columns:
        lc = c.lower()
        if any(k in lc for k in ["indicator","kpi","metric","item","parameter"]) and indicator_col is None:
            indicator_col = c
        if any(k in lc for k in ["value","count","coverage","percentage","percent","rate"]) and value_col is None:
            if _is_numeric_series(df[c], min_non_na=0.2):
                value_col = c
    if indicator_col and value_col:
        # ---------------- LONG ----------------
        w = pd.DataFrame()
        w["state"]     = df[id_map["state"]].astype(str).str.strip() if "state" in id_map else "India"
        w["district"]  = df[id_map["district"]].astype(str).str.strip() if "district" in id_map else np.nan
        w["indicator"] = df[indicator_col].astype(str).str.strip()
        w["value"]     = pd.to_numeric(df[value_col], errors="coerce")
        w["unit"]      = (df[id_map["unit"]] if "unit" in id_map else "")
        # Date
        if "year" in id_map and "month" in id_map:
            y = pd.to_numeric(df[id_map["year"]], errors="coerce")
            m = df[id_map["month"]].apply(parse_month)
            mask = (~y.isna()) & (~m.isna())
            w = w[mask].copy()
            w["date"] = [to_month_start(int(yy), int(mm)) for yy,mm in zip(y[mask].astype(int), m[mask].astype(int))]
        elif "date" in id_map:
            w["date"] = pd.to_datetime(df[id_map["date"]], errors="coerce").dt.to_period("M").dt.to_timestamp()
        else:
            raise ValueError("Could not locate (year, month) or a monthly 'date' column.")
        w = w.dropna(subset=["value","date"]).reset_index(drop=True)
        tidy = w[["date","state","district","indicator","value","unit"]]
        mode = "LONG"
    else:
        # ---------------- WIDE → LONG ----------------
        # Build id columns that exist
        id_cols = [id_map[k] for k in ["state","district","year","month","date","unit"] if k in id_map]
        id_cols = list(dict.fromkeys(id_cols))  # unique & ordered

        # Candidate metric columns (mostly numeric)
        metric_cols = []
        for c in df.columns:
            if c in id_cols: 
                continue
            lc = c.lower()
            if any(k in lc for k in ["code","id","remarks","remark","comment","source"]):
                continue
            if _is_numeric_series(df[c], min_non_na=0.2):
                metric_cols.append(c)
        if not metric_cols:
            print("Columns found:", list(df.columns))
            raise ValueError("Could not detect metric columns to melt. Please tailor loader mapping.")

        long_df = df.melt(
            id_vars=id_cols,
            value_vars=metric_cols,
            var_name="indicator",
            value_name="value"
        )
        long_df["state"]    = long_df[id_map["state"]].astype(str).str.strip() if "state" in id_map else "India"
        long_df["district"] = long_df[id_map["district"]].astype(str).str.strip() if "district" in id_map else np.nan
        # Date
        if "year" in id_map and "month" in id_map:
            y = pd.to_numeric(long_df[id_map["year"]], errors="coerce")
            m = long_df[id_map["month"]].apply(parse_month)
            mask = (~y.isna()) & (~m.isna())
            long_df = long_df[mask].copy()
            long_df["date"] = [to_month_start(int(yy), int(mm)) for yy,mm in zip(y[mask].astype(int), m[mask].astype(int))]
        elif "date" in id_map:
            long_df["date"] = pd.to_datetime(long_df[id_map["date"]], errors="coerce").dt.to_period("M").dt.to_timestamp()
        else:
            raise ValueError("Could not locate (year, month) or a monthly 'date' column.")

        # Unit: use provided column or infer per-indicator from values
        if "unit" in id_map:
            long_df["unit"] = long_df[id_map["unit"]]
        else:
            long_df["unit"] = ""
            for ind, g in long_df.groupby("indicator"):
                long_df.loc[g.index, "unit"] = _infer_unit_from_indicator_and_values(ind, g["value"])

        tidy = (long_df[["date","state","district","indicator","value","unit"]]
                .dropna(subset=["value","date"])
                .reset_index(drop=True))
        mode = "WIDE→LONG"

    print(f"[OK] Loaded Poshan ({mode}) | rows: {len(tidy):,}")
    print("Sample indicators:", tidy["indicator"].dropna().unique()[:8])
    return tidy

t0 = time.time()  # start timing the whole pipeline for decision-speed
poshan = load_poshan_any(DATA_PATH)
display(poshan.head(10))

[OK] Loaded Poshan (WIDE→LONG) | rows: 122,310
Sample indicators: ['awc_infra_dws' 'awc_infra_fun_toilets' 'awc_infra_own_buil'
 'aadhar_verf_benf' 'adolescent_girls' 'awc_open_1day' 'awc_open_21day'
 'aww']


,date,state,district,indicator,value,unit
0,2023-01-01,Andaman And Nicobar Islands,North And Middle Andaman,awc_infra_dws,230,
1,2023-01-01,Andaman And Nicobar Islands,Nicobars,awc_infra_dws,53,
2,2023-01-01,Andaman And Nicobar Islands,South Andamans,awc_infra_dws,340,
3,2023-01-01,Andhra Pradesh,Anantapur,awc_infra_dws,1160,
4,2023-01-01,Andhra Pradesh,Alluri Sitharama Raju,awc_infra_dws,0,
5,2023-01-01,Andhra Pradesh,Anakapalli,awc_infra_dws,0,
6,2023-01-01,Andhra Pradesh,Annamayya,awc_infra_dws,0,
7,2023-01-01,Andhra Pradesh,Bapatla,awc_infra_dws,0,
8,2023-01-01,Andhra Pradesh,Chittoor,awc_infra_dws,825,
9,2023-01-01,Andhra Pradesh,East Godavari,awc_infra_dws,1058,


In [17]:
# ======================================================================
# CELL 5 — Canonical KPI mapping (regex patterns → KPI keys)
# ======================================================================
CANONICAL_PATTERNS = [
    (r"full.*immun",                 "full_immunization_pct"),
    (r"\banc.*\b4\b|\b4\+.*anc",     "anc4_pct"),
    (r"institutional.*deliver",      "institutional_delivery_pct"),
    (r"\bawc.*open.*day",            "awc_open_days"),
    (r"growth.*monitor",             "growth_monitoring_pct"),
    (r"\bthr\b.*distribut",          "thr_distribution_pct"),
    (r"wasting",                     "wasting_pct"),
    (r"stunting",                    "stunting_pct"),
    (r"\bsam\b|severe.*acute.*mal",  "sam_pct"),
    (r"\bmam\b|moderate.*acute.*mal","mam_pct"),
]

def map_indicator_to_kpi(ind):
    s = str(ind).strip().lower()
    for pat, kpi in CANONICAL_PATTERNS:
        if re.search(pat, s, flags=re.IGNORECASE):
            return kpi
    return ind  # fallback to original label if no pattern match

std = poshan.copy()
std["kpi"] = std["indicator"].apply(map_indicator_to_kpi)
std["is_pct"] = std["unit"].astype(str).str.contains("%", case=False, na=False) | std["kpi"].str.contains("pct")
std = std.sort_values(["state","district","kpi","date"]).reset_index(drop=True)

display(std.head(8))
print("KPIs found (sample):", std["kpi"].unique()[:12])

,date,state,district,indicator,value,unit,kpi,is_pct
0,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,447,,aadhar_verf_benf,False
1,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,450,,aadhar_verf_benf,False
2,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,406,,aadhar_verf_benf,False
3,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,420,,aadhar_verf_benf,False
4,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,433,,aadhar_verf_benf,False
5,2023-01-01,Andaman And Nicobar Islands,Nicobars,aadhar_verf_benf,441,,aadhar_verf_benf,False
6,2023-01-01,Andaman And Nicobar Islands,Nicobars,adolescent_girls,0,%,adolescent_girls,True
7,2023-01-01,Andaman And Nicobar Islands,Nicobars,adolescent_girls,0,%,adolescent_girls,True


KPIs found (sample): ['aadhar_verf_benf' 'adolescent_girls' 'awc' 'awc_infra_dws'
 'awc_infra_fun_toilets' 'awc_infra_own_buil' 'awc_open_days' 'aww'
 'children_3_6_years' 'children_3_6_years_nonschool'
 'children_3_6_years_school' 'children_6months_3years']


In [18]:
# ======================================================================
# CELL 6 — Aggregations, deltas and rule-based flags (for action queue)
# ======================================================================
# Prefer district granularity if present; fallback to state
has_districts = std["district"].notna().any()

by_dist  = std.groupby(["date","state","district","kpi"], as_index=False)["value"].mean()
by_state = std.groupby(["date","state","kpi"], as_index=False)["value"].mean().assign(district=np.nan)
kpi_df = by_dist if has_districts else by_state

# Deltas
kpi_df = kpi_df.sort_values(["state","district","kpi","date"]).reset_index(drop=True)
kpi_df["value_lag1"]  = kpi_df.groupby(["state","district","kpi"])["value"].shift(1)
kpi_df["mom_delta"]   = kpi_df["value"] - kpi_df["value_lag1"]
kpi_df["value_lag12"] = kpi_df.groupby(["state","district","kpi"])["value"].shift(12)
kpi_df["yoy_delta"]   = kpi_df["value"] - kpi_df["value_lag12"]

def flag_rule(row):
    kpi = row["kpi"]; val = row["value"]
    if pd.isna(val): 
        return False
    thr = KPI_THRESHOLDS.get(kpi)
    if thr is None: 
        return False
    # Coverage: flag if below target
    if kpi in ("full_immunization_pct","anc4_pct","institutional_delivery_pct",
               "thr_distribution_pct","growth_monitoring_pct"):
        return val < thr
    # Undernutrition: flag if above threshold
    if kpi in ("wasting_pct","stunting_pct","sam_pct","mam_pct"):
        return val > thr
    # Operations
    if kpi == "awc_open_days":
        return val < thr
    return False

kpi_df["flag"] = kpi_df.apply(flag_rule, axis=1)

latest_date = kpi_df["date"].max()
action_queue = (kpi_df[kpi_df["date"] == latest_date]
                .loc[kpi_df["flag"]]
                .sort_values(["state","district","kpi"]))
print(f"Latest month in data: {latest_date.date()} | Flags: {len(action_queue)}")
display(action_queue.head(20))

ACTION_CSV = OUT_DIR / f"action_queue_{latest_date:%Y_%m}.csv"
action_queue.to_csv(ACTION_CSV, index=False)
print("Saved action queue →", ACTION_CSV)

Latest month in data: 2023-01-01 | Flags: 16


,date,state,district,kpi,value,value_lag1,mom_delta,value_lag12,yoy_delta,flag
84,2023-01-01,Andhra Pradesh,Alluri Sitharama Raju,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
110,2023-01-01,Andhra Pradesh,Anakapalli,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
162,2023-01-01,Andhra Pradesh,Annamayya,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
188,2023-01-01,Andhra Pradesh,Bapatla,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
344,2023-01-01,Andhra Pradesh,Konaseema,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
448,2023-01-01,Andhra Pradesh,Ntr,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
474,2023-01-01,Andhra Pradesh,Palnadu,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
500,2023-01-01,Andhra Pradesh,Parvathipuram Manyam,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True
760,2023-01-01,Arunachal Pradesh,Anjaw,awc_open_days,18.333333,NaN,NaN,NaN,NaN,True
812,2023-01-01,Arunachal Pradesh,Dibang Valley,awc_open_days,0.000000,NaN,NaN,NaN,NaN,True


Saved action queue → C:\Users\JOSHUVA\anaconda_projects\3066d52d-ee81-4d66-a621-33a9ced0caf0\outputs\action_queue_2023_01.csv


In [19]:
# ======================================================================
# CELL 7 — Interactive mini-dashboard (in-notebook)
# ======================================================================
states = sorted(kpi_df["state"].dropna().unique().tolist())
kpis   = sorted(kpi_df["kpi"].dropna().unique().tolist())

def plot_state(state, kpi_key):
    scope = kpi_df[(kpi_df["state"]==state) & (kpi_df["kpi"]==kpi_key)]
    if scope.empty:
        fig = go.Figure().add_annotation(text="No data", showarrow=False)
        return fig, None

    title = f"{kpi_key} — {state}"

    # Time series (state mean; if districts exist, still show state-level mean over districts)
    ts = scope.groupby("date", as_index=False)["value"].mean()
    fig_ts = px.line(ts, x="date", y="value", markers=True,
                     title=f"{title} (time series)",
                     labels={"value":"Value","date":"Month"})
    fig_ts.update_layout(height=380)

    # Latest month by district (if available)
    fig_bar = None
    if has_districts:
        rec = scope[scope["date"]==scope["date"].max()]
        if not rec.empty:
            fig_bar = px.bar(rec, x="district", y="value",
                             title=f"{title} (latest month by district)",
                             labels={"value":"Value","district":"District"})
            fig_bar.update_layout(height=420)
    return fig_ts, fig_bar

print("Use the dropdowns to explore:")
@interact(State=Dropdown(options=states, description="State:", layout={'width':'300px'}),
          KPI=Dropdown(options=kpis, description="KPI:", layout={'width':'300px'}))
def _dash(State, KPI):
    fig_ts, fig_bar = plot_state(State, KPI)
    fig_ts.show()
    if fig_bar: fig_bar.show()

Use the dropdowns to explore:


interactive(children=(Dropdown(description='State:', layout=Layout(width='300px'), options=('Andaman And Nicob…